In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
#python setup_database.py & python setup_qdrant.py & python setup_elastic.py & python setup_redis.py

In [34]:
import sys
sys.path.append('/app')

In [35]:

from api_client import APIClient
from uuid import UUID
import asyncio
from datetime import datetime, timedelta

In [36]:

client = await APIClient.create(email="2280905@gmail.com")

In [37]:
client

In [38]:

# Auth examples
#auth_result = await client.google_auth("google_token_here", {"utm_source": "web"})



In [39]:
token_result = await client.submit_token("3ae04e20124d40babc5107e658c666b6")
    


In [40]:
token_result

{'user_id': 'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
 'user_name': 'Dmitriy Grankin',
 'image': None}

In [41]:
# Content management
# Get contents with filters
contents = await client.get_contents(
    content_type="meeting",
    filters=[{
       "type": "speaker",
       "values": ["Dmitry Grankin"]
    }],
    offset=0,
    limit=20
)

contents



#failed - filter does not work

{'total': 48,
 'contents': [{'content_id': '081d04c6-827d-4375-af68-9f8fbd50b707',
   'type': 'meeting',
   'text': None,
   'timestamp': '2024-09-10T12:01:58',
   'is_indexed': False,
   'is_owner': True,
   'access_level': 'owner',
   'entities': [{'name': 'Dmitry Grankin', 'type': 'speaker'}]},
  {'content_id': '0aee5d35-5e04-4b48-b428-6e2979aab55c',
   'type': 'meeting',
   'text': None,
   'timestamp': '2024-09-25T13:28:47',
   'is_indexed': False,
   'is_owner': True,
   'access_level': 'owner',
   'entities': [{'name': 'Dmitry Grankin', 'type': 'speaker'}]},
  {'content_id': '1fb0bd86-3ca7-41bb-8561-f56586a5dadd',
   'type': 'meeting',
   'text': None,
   'timestamp': '2025-01-08T16:04:35',
   'is_indexed': False,
   'is_owner': True,
   'access_level': 'owner',
   'entities': [{'name': 'Dmitry Grankin', 'type': 'speaker'}]},
  {'content_id': '2182def0-c078-45f6-bce1-cbac254c8007',
   'type': 'meeting',
   'text': None,
   'timestamp': '2024-12-05T11:02:05',
   'is_indexed': Fal

In [42]:
# Add new content
content = await client.add_content(
    body="Sample content text",
    content_type="note",
    entities=[{"type": "tag", "name": "test"}]
)


In [43]:
content

{'content_id': '10f8dbae-e76a-4b59-b6db-1ef9a381c4d9'}

In [44]:
content_id = UUID(content["content_id"])

# Get single content
single_content = await client.get_content(content_id)
single_content

{'content_id': '10f8dbae-e76a-4b59-b6db-1ef9a381c4d9',
 'type': 'note',
 'text': 'Sample content text',
 'timestamp': '2025-01-28T19:22:05.475252',
 'is_indexed': False,
 'access_level': 'owner',
 'is_owner': True,
 'entities': [{'name': 'test', 'type': 'tag'}],
 'parent_id': None,
 'meta': None}

In [45]:
# Modify content
modified = await client.modify_content(
    content_id,
    body="Updated content text",
    entities=[{"type": "speaker", "name": "Jane Doe"}]
)


In [46]:
modified

{'success': True}

In [47]:
# Get single content
single_content = await client.get_content(content_id)
single_content



{'content_id': '10f8dbae-e76a-4b59-b6db-1ef9a381c4d9',
 'type': 'note',
 'text': 'Updated content text',
 'timestamp': '2025-01-28T19:22:05.475252',
 'is_indexed': False,
 'access_level': 'owner',
 'is_owner': True,
 'entities': [{'name': 'Jane Doe', 'type': 'speaker'}],
 'parent_id': None,
 'meta': None}

In [48]:
archived = await client.archive_content(content_id)

In [49]:
archived

{'success': True}

In [50]:
content_id

UUID('10f8dbae-e76a-4b59-b6db-1ef9a381c4d9')

In [51]:
try:
    single_content = await client.get_content(content_id)
except Exception as e:
    print(e)


#OK - content unavailable after archive

Content not found or no access


In [52]:
restored = await client.restore_content(content_id)
single_content = await client.get_content(content_id)
single_content

{'content_id': '10f8dbae-e76a-4b59-b6db-1ef9a381c4d9',
 'type': 'note',
 'text': 'Updated content text',
 'timestamp': '2025-01-28T19:22:05.475252',
 'is_indexed': False,
 'access_level': 'owner',
 'is_owner': True,
 'entities': [{'name': 'Jane Doe', 'type': 'speaker'}],
 'parent_id': None,
 'meta': None}

In [53]:

try:
    deleted = await client.delete_content(content_id)
except Exception as e:
    print(e)
try:
    single_content = await client.get_content(content_id)
except Exception as e:
    print(e)
    
#OK - content unavailable after delete

Content not found or no access


In [54]:

try:
    restored = await client.restore_content(content_id)
except Exception as e:
    print(e)
try:
    single_content = await client.get_content(content_id)
except Exception as e:
    print(e)

#OK - content unavailable after delete

Content not found or not archived


In [55]:
# Add new content
content = await client.add_content(
    body="Sample content text",
    content_type="note",
    entities=[{"type": "speaker", "name": "John Doe"}]
)
content

{'content_id': '1b413e00-9c15-4832-8c0f-be99861b0586'}

In [56]:
# Sharing
share_link = await client.create_share_link(
    access_level="transcript",
    content_ids=[content_id],
    target_email="recipient@example.com",
    expiration_hours=24
)
    


In [57]:
accepted = await client.accept_share_link(
    token=share_link["token"],
    accepting_email="recipient@example.com"
)


In [58]:

# Index content
indexed = await client.index_content(content_id)
indexed



{'status': 'queued', 'message': 'Content has been queued for indexing'}

In [59]:
# Entity management
entities = await client.get_entities("speaker", offset=0, limit=20)
entities


[{'type': 'speaker', 'name': 'John Doe', 'id': 5704},
 {'type': 'speaker', 'name': 'Jane Doe', 'id': 5705}]

In [60]:
# Entity management
entities = await client.get_entities("tag", offset=0, limit=20)
entities


[{'type': 'tag', 'name': 'AI', 'id': 5729},
 {'type': 'tag', 'name': 'test', 'id': 5722}]

In [61]:
# Chat interactions
async for response in client.chat(
    query="What is this document about?",
    content_id=content_id,
    model="gpt-4o-mini",
    temperature=0.7
):
    print(response)



{'chunk': 'It'}
{'chunk': ' seems'}
{'chunk': ' there'}
{'chunk': ' is'}
{'chunk': ' no'}
{'chunk': ' specific'}
{'chunk': ' document'}
{'chunk': ' provided'}
{'chunk': ' for'}
{'chunk': ' me'}
{'chunk': ' to'}
{'chunk': ' analyze'}
{'chunk': '.'}
{'chunk': ' If'}
{'chunk': ' you'}
{'chunk': ' could'}
{'chunk': ' share'}
{'chunk': ' the'}
{'chunk': ' document'}
{'chunk': ' or'}
{'chunk': ' provide'}
{'chunk': ' more'}
{'chunk': ' details'}
{'chunk': ' about'}
{'chunk': ' its'}
{'chunk': ' content'}
{'chunk': ','}
{'chunk': ' I'}
{'chunk': ' would'}
{'chunk': ' be'}
{'chunk': ' happy'}
{'chunk': ' to'}
{'chunk': ' help'}
{'chunk': ' you'}
{'chunk': ' understand'}
{'chunk': ' what'}
{'chunk': ' it'}
{'chunk': ' is'}
{'chunk': ' about'}
{'chunk': '!'}
{'thread_id': 'fe1307f9-0216-489f-8440-f9643cb96f17', 'content_id': '10f8dbae-e76a-4b59-b6db-1ef9a381c4d9', 'entity_id': None, 'content_ids': None, 'entity_ids': None, 'output': 'It seems there is no specific document provided for me to anal

In [ ]:
# Edit chat message
async for response in client.edit_chat_message(
    thread_id="thread_123",
    message_index=1,
    new_content="Updated message content",
    model="gpt-4o-mini"
):
    print(response["content"])

# Thread management
threads = await client.get_threads(
    content_id=content_id,
    only_archived=False,
    limit=50,
    start_date=datetime.now() - timedelta(days=7)
)

thread = await client.get_thread("thread_123")
archived_thread = await client.archive_thread("thread_123")
restored_thread = await client.restore_thread("thread_123")
renamed = await client.rename_thread("thread_123", "New Thread Name")
